In [20]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer,EarlyStoppingCallback
from sklearn.model_selection import ParameterGrid
from src.utils.myutils import *
import yaml
import json

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

BATCH_SIZE = 64
transformers.utils.logging.set_verbosity_error()

In [21]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Using custom data configuration default-f28f8af5b44ab214
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-f28f8af5b44ab214/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [22]:
def fit_and_eval(model_name,token_full,train_idx,eval_idx,training_args):
    token_train = Dataset.from_dict(token_full[train_index])
    token_valid = Dataset.from_dict(token_full[val_index])
    
    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.to(device);
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
    
    trainer.train();
    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator);
    return compute_metrics(model,device,eval_dataloader)['f1']
    

In [23]:
model_name = config_data['model_to_tune'][0]
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
token_full = preprocess_data(data,tokenizer,'sentence')

Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-f28f8af5b44ab214/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-74b152a0ca0f6626.arrow


In [24]:
model_name

'ufal/robeczech-base'

In [28]:
str(params)

"{'batch_size': 32, 'epochs': 4, 'learning_rate': 5e-05, 'warmup_steps': 0, 'weight_decay': 0.1}"

In [ ]:
param_grid = {
    'learning_rate':[2e-5,3e-5,5e-5],
    'weight_decay':[0.1],
    'batch_size': [16, 32],
    'warmup_steps': [0],
    'epochs':[2,3,4]
}
param_comb = ParameterGrid(param_grid)

with open(PATH + '/src/models/robe_hyperparam_search.txt','w') as f:
    for idx,params in enumerate(param_comb):

        BATCH_SIZE=params['batch_size']
        training_args = TrainingArguments(
            output_dir = './',
            num_train_epochs=params['epochs'],
            save_total_limit=2,
            disable_tqdm=False,
            per_device_train_batch_size=params['batch_size'],  
            warmup_steps=params['warmup_steps'],
            weight_decay=params['weight_decay'],
            learning_rate=params['learning_rate'])

        scores = []
        print("Fitting model on params #",idx)

        #run 5-fold CV
        for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

            scores.append(fit_and_eval(model_name,token_full,train_index,val_index,training_args));

        f.write(json.dumps(params))
        f.write(str(np.mean(scores)))
        f.write("\n\n")


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do thi

Fitting model on params # 0


loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 51961
}

loading weights file https://huggingface.co/ufal/robeczech-base/resolve/m

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

...

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

Step,Training Loss


In [10]:
scores

[0.7696, 0.7792, 0.7580128205128205, 0.7676282051282052, 0.7724358974358975]

In [15]:
params

{'batch_size': 32,
 'learning_rate': 2e-05,
 'warmup_steps': 50,
 'weight_decay': 0.1}

In [26]:
with open(PATH + '/src/models/hyperparam_search.txt','w') as f:
    for i in range(2):
        f.write(json.dumps(params))
        f.write(" f1: " + str(np.mean(scores))+"\n")
        

In [13]:
from tqdm import tqdm

In [14]:
model_scores = {}

for model_name in tqdm(config_data['models']):
    model_name = 'fav-kky/FERNET-C5'
    
    BATCH_SIZE=16
    training_args = TrainingArguments(
            output_dir = './',
            num_train_epochs=2,
            save_total_limit=2,
            disable_tqdm=False,
            per_device_train_batch_size=BATCH_SIZE,  
            warmup_steps=0,
            weight_decay=0.1,
            learning_rate=3e-5)
    
    scores = []
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    token_full = preprocess_data(data,tokenizer,'sentence')

    print("Running 5-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])


        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        model.to(device)
        trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
        trainer.train()

        #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
    print("Done.")
    model_scores[model_name] = scores


  0%|          | 0/6 [00:00<?, ?it/s]PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/3d9cd6327fe71a900f5b330c7ba118b1f0e0e64909942cb51846a9a9ebd1da4d.4041320f90f70c06edf95880a7a45a318565ef4291c71434f29adbd4aea0eae5
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


loading configuration file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/3d9cd6327fe71a900f5b330c7ba118b1f0e0e64909942cb51846a9a9ebd1da4d.4041320f90f70c06edf95880a7a45a318565ef4291c71434f29adbd4aea0eae5
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/pytorch_model.bin from cache at /home/horyctom/.ca

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/3d9cd6327fe71a900f5b330c7ba118b1f0e0e64909942cb51846a9a9ebd1da4d.4041320f90f70c06edf95880a7a45a318565ef4291c71434f29adbd4aea0eae5
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size"

...

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/fav-kky/FERNET-C5/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/3d9cd6327fe71a900f5b330c7ba118b1f0e0e64909942cb51846a9a9ebd1da4d.4041320f90f70c06edf95880a7a45a318565ef4291c71434f29adbd4aea0eae5
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size"

Step,Training Loss


 17%|█▋        | 1/6 [04:43<23:36, 283.27s/it]


KeyboardInterrupt: 

In [17]:
np.mean(model_scores['fav-kky/FERNET-C5'])

0.7860220512820513